In [0]:
%pip install -q pyspark==3.1.2 spark-nlp

Python interpreter will be restarted.
Python interpreter will be restarted.

In [0]:
import json
import pandas as pd
import numpy as np
# Import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Import SparkNLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *


spark = SparkSession.builder.config('spark.driver.maxResultSize', '18g').getOrCreate()
data_path = "dbfs:/mnt/lsde/group05/comments_sample/2020.parquet"
df =spark.read.load(data_path,format="parquet")

In [0]:
df.show()

+---------+---------------+--------------------+-------+-----+----+----------------+------------+-----------+
parent_id| author| body| id|score| ups| subreddit|subreddit_id|created_utc|
+---------+---------------+--------------------+-------+-----+----+----------------+------------+-----------+
t3_htorhu|ProodsthePriest|I knew it was cak...|fyiy1jg| 2|null|nextfuckinglevel| t5_m0bnr| 1595129067|
t3_htr0hw| thetrollbot200|Im glad we’re not...|fyj1ieg| 1|null| worldnews| t5_2qh13| 1595131523|
t3_htrrpz| no-such-file|Buckwheat kasha. ...|fyj92zw| 21|null| AskARussian| t5_2xzpo| 1595137529|
t3_htkhxe| PorannaSztyca| Welcome to Russia|fyi3jq1| 2|null| pics| t5_2qh0u| 1595109983|
t3_htogvp| veganmark| LOL!!!|fyi3sa7| 13|null| WayOfTheBern| t5_3ff8l| 1595110120|
t3_htovo6| discobrisco|Tbh that’s more t...|fyi75he| 449|null| MemeEconomy| t5_3gl3k| 1595112056|
t3_htkhxe| dukemagnus|Tabula rasa.. a n...|fyidlop| 1|null| pics| t5_2qh0u| 1595115818|
t3_hs39g0| CmdrBlindman|Have you guys tri...|fy8k2jr| 2|null| darksouls3| t5_2w3z3| 1594892195|
t3_hl4qmi| xotonic|Most Russians wat...|fwy11lz| 3|null| AskARussian| t5_2xzpo| 1593901315|
t3_hlbs4n| Chakawatts|The SR-3 has a bu...|fwy1bqf| 1|null| modernwarfare| t5_2r8qf| 1593901484|
t3_hytam5| APinkSlime677|There just needs ...|fzg9d4v| 1|null| Eyebleach| t5_2s427| 1595888019|
t3_hsauyv| Skatingraccoon|The general respo...|fy99n5i| 6|null| answers| t5_2qkeh| 1594910791|
t3_hsamoq| autotldr|This is the best ...|fy9iwco| 1|null| politics| t5_2cneq| 1594915411|
t3_hs8fwc| Hisgenart|It looks like the...|fy9o1h4| 8|null| cursedfoods| t5_o02vq| 1594917892|
t3_hrt1ha| ArdenSix|Going to have a l...|fy9oq5h| 5|null| worldnews| t5_2qh13| 1594918221|
t3_fypu3c| Kvirin_| Дарова, Россияне|fn17x3o| 1|null| TheYouShow| t5_2dptgf| 1586556565|
t3_fypu3c| megavolt356| Play megalovania|fn183nc| 1|null| TheYouShow| t5_2dptgf| 1586556668|
t3_fypu3c|kiravonconcrete|Spasiba from New ...|fn18b3t| 1|null| TheYouShow| t5_2dptgf| 1586556788|
t3_fypu3c| Demstillers7|This is so fuckin...|fn18b8k| 1|null| TheYouShow| t5_2dptgf| 1586556790|
t3_fypu3c| issie78| respect my cryipin|fn18gvd| 1|null| TheYouShow| t5_2dptgf| 1586556880|
+---------+---------------+--------------------+-------+-----+----+----------------+------------+-----------+
only showing top 20 rows

In [0]:
data =  df.toPandas()

In [0]:
data

Out[4]:

,parent_id,author,body,id,score,ups,subreddit,subreddit_id,created_utc
0,t3_fypu3c,Kvirin_,"Дарова, Россияне",fn17x3o,1,NaN,TheYouShow,t5_2dptgf,1586556565
1,t3_fypu3c,megavolt356,Play megalovania,fn183nc,1,NaN,TheYouShow,t5_2dptgf,1586556668
2,t3_fypu3c,kiravonconcrete,Spasiba from New Jersey USA with a juice glass...,fn18b3t,1,NaN,TheYouShow,t5_2dptgf,1586556788
3,t3_fypu3c,Demstillers7,This is so fucking surreal,fn18b8k,1,NaN,TheYouShow,t5_2dptgf,1586556790
4,t3_fypu3c,issie78,respect my cryipin,fn18gvd,1,NaN,TheYouShow,t5_2dptgf,1586556880
...,...,...,...,...,...,...,...,...,...
175,t3_k0bsgn,thefriedgalaxy,I know braithwaite scored two but IMO dest motm,gdhlwzw,27,NaN,Barca,t5_2s561,1606254825
176,t3_jzmxdh,mostitostedium,"Plot twist, they leave the store and the other...",gddhee5,20,NaN,ANormalDayInRussia,t5_2z166,1606168212
177,t3_jmqalr,SecondWorld1198,That's it. The greatest title ever.,gaxnu3b,0,NaN,news,t5_2qh3l,1604350149
178,t3_jmw76b,xyphocrystal,Holy what if :D,gaxqp2m,1,NaN,pics,t5_2qh0u,1604351517


In [0]:
data["position"] = -1

In [0]:
data

Out[6]:

,parent_id,author,body,id,score,ups,subreddit,subreddit_id,created_utc,position
0,t3_fypu3c,Kvirin_,"Дарова, Россияне",fn17x3o,1,NaN,TheYouShow,t5_2dptgf,1586556565,-1
1,t3_fypu3c,megavolt356,Play megalovania,fn183nc,1,NaN,TheYouShow,t5_2dptgf,1586556668,-1
2,t3_fypu3c,kiravonconcrete,Spasiba from New Jersey USA with a juice glass...,fn18b3t,1,NaN,TheYouShow,t5_2dptgf,1586556788,-1
3,t3_fypu3c,Demstillers7,This is so fucking surreal,fn18b8k,1,NaN,TheYouShow,t5_2dptgf,1586556790,-1
4,t3_fypu3c,issie78,respect my cryipin,fn18gvd,1,NaN,TheYouShow,t5_2dptgf,1586556880,-1
...,...,...,...,...,...,...,...,...,...,...
175,t3_k0bsgn,thefriedgalaxy,I know braithwaite scored two but IMO dest motm,gdhlwzw,27,NaN,Barca,t5_2s561,1606254825,-1
176,t3_jzmxdh,mostitostedium,"Plot twist, they leave the store and the other...",gddhee5,20,NaN,ANormalDayInRussia,t5_2z166,1606168212,-1
177,t3_jmqalr,SecondWorld1198,That's it. The greatest title ever.,gaxnu3b,0,NaN,news,t5_2qh3l,1604350149,-1
178,t3_jmw76b,xyphocrystal,Holy what if :D,gaxqp2m,1,NaN,pics,t5_2qh0u,1604351517,-1


In [0]:
#0:ProUkriane; 1:ProRussia; 2:not relative
for i in range(len(data["body"])):
    print(str(i)+"  ######"+data["subreddit"][i]+"######")
    data["body"][i]

0 ######TheYouShow######
Дарова, Россияне: